In [1]:
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil
import numpy as np
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.estimators.kmeans import H2OKMeansEstimator
from h2o.estimators.pca import H2OPrincipalComponentAnalysisEstimator

In [2]:
# display matplotlib graphics in notebook
%matplotlib inline

In [3]:
pct_memory=0.5
virtual_memory=psutil.virtual_memory()
min_mem_size=int(round(int(pct_memory*virtual_memory.available)/1073741824,0))
print(min_mem_size)

3


In [4]:
# 65535 Highest port no
port_no=random.randint(5555,55555)
h2o.init(strict_version_check=False,min_mem_size_GB=min_mem_size,port=port_no) # start h2o

Checking whether there is an H2O instance running at http://localhost:50429..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_121"; OpenJDK Runtime Environment (Zulu 8.20.0.5-macosx) (build 1.8.0_121-b15); OpenJDK 64-Bit Server VM (Zulu 8.20.0.5-macosx) (build 25.121-b15, mixed mode)
  Starting server from /Users/bear/anaconda/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/lh/42j8mfjx069d1bkc2wlf2pw40000gn/T/tmpq_fc7l4p
  JVM stdout: /var/folders/lh/42j8mfjx069d1bkc2wlf2pw40000gn/T/tmpq_fc7l4p/h2o_bear_started_from_python.out
  JVM stderr: /var/folders/lh/42j8mfjx069d1bkc2wlf2pw40000gn/T/tmpq_fc7l4p/h2o_bear_started_from_python.err
  Server is running at http://127.0.0.1:50429
Connecting to H2O server at http://127.0.0.1:50429... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.3
H2O cluster version age:,"14 days, 4 hours and 1 minute"
H2O cluster name:,H2O_from_python_bear_wgcfo2
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [5]:
# load clean data
path = 'data/loan.csv'

In [6]:
# define input variable measurement levels 
# strings automatically parsed as enums (nominal)
# numbers automatically parsed as numeric
col_types = {'bad_loan': 'enum'}

In [7]:
frame = h2o.import_file(path=path, col_types=col_types) # multi-threaded import

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [8]:
frame.describe()

Rows:163987
Cols:15




,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,purpose,addr_state,dti,delinq_2yrs,revol_util,total_acc,bad_loan,longest_credit_length,verification_status
type,int,enum,real,int,enum,real,enum,enum,real,int,real,int,enum,int,enum
mins,500.0,,5.42,0.0,,1896.0,,,0.0,0.0,0.0,1.0,,0.0,
mean,13074.169141456336,,13.715904065566173,5.68435293299533,,71915.67051974901,,,15.881530121290117,0.2273570060625282,54.07917280242258,24.579733834274638,,14.854273655448353,
maxs,35000.0,,26.06,10.0,,7141778.0,,,39.99,29.0,150.70000000000002,118.0,,65.0,
sigma,7993.556188734649,,4.391939870545795,3.6106637311002365,,59070.915654918244,,,7.587668224192549,0.6941679229284182,25.285366766770505,11.685190365910659,,6.947732922546696,
zeros,0,,0,14248,,0,,,270,139459,1562,0,,11,
missing,0,0,0,5804,0,4,0,0,0,29,193,29,0,29,0
0,5000.0,36 months,10.65,10.0,RENT,24000.0,credit_card,AZ,27.65,0.0,83.7,9.0,0,26.0,verified
1,2500.0,60 months,15.27,0.0,RENT,30000.0,car,GA,1.0,0.0,9.4,4.0,1,12.0,verified
2,2400.0,36 months,15.96,10.0,RENT,12252.0,small_business,IL,8.72,0.0,98.5,10.0,0,10.0,not verified


In [9]:
# assign target and inputs
y = 'bad_loan'
X = [name for name in frame.columns if name != y]
print(y)
print(X)

bad_loan
['loan_amnt', 'term', 'int_rate', 'emp_length', 'home_ownership', 'annual_inc', 'purpose', 'addr_state', 'dti', 'delinq_2yrs', 'revol_util', 'total_acc', 'longest_credit_length', 'verification_status']


In [10]:
# determine column types
reals, enums = [], []
for key, val in frame.types.items():
    if key in X:
        if val == 'enum':
            enums.append(key)
        else: 
            reals.append(key)

print(enums)
print(reals)

['term', 'home_ownership', 'purpose', 'addr_state', 'verification_status']
['loan_amnt', 'int_rate', 'emp_length', 'annual_inc', 'dti', 'delinq_2yrs', 'revol_util', 'total_acc', 'longest_credit_length']


In [11]:
# impute missing values
# standardize for both clustering and PCA
frame[reals].impute(method='mean')
frame[reals] = frame[reals].scale()

In [12]:
# train k-means cluster model
# print summary
clusters = H2OKMeansEstimator(standardize=False, k=3, seed=12345)
clusters.train(x=X, training_frame=frame)
print(clusters)

# view detailed results at http://ip:port/flow/index.html

kmeans Model Build progress: |████████████████████████████████████████████| 100%
Model Details
H2OKMeansEstimator :  K-means
Model Key:  KMeans_model_python_1549687847146_1


ModelMetricsClustering: kmeans
** Reported on train data. **

MSE: NaN
RMSE: NaN
Total Within Cluster Sum of Square Error: 1597695.0117686181
Total Sum of Square Error to Grand Mean: 1859349.995340903
Between Cluster Sum of Square Error: 261654.98357228492
Centroid Statistics: 


,centroid,size,within_cluster_sum_of_squares
,1.0,102044.0,848132.4869012
,2.0,6.0,1899.4841206
,3.0,61937.0,747663.0407469


Scoring History: 


,timestamp,duration,iterations,number_of_reassigned_observations,within_cluster_sum_of_squares
,2019-02-08 23:50:52,0.019 sec,0.0,nan,nan
,2019-02-08 23:50:53,0.755 sec,1.0,163987.0,3748306.5105250
,2019-02-08 23:50:53,1.023 sec,2.0,217.0,1775427.0474256
,2019-02-08 23:50:53,1.065 sec,3.0,411.0,1771667.8630573
,2019-02-08 23:50:53,1.101 sec,4.0,747.0,1766087.4662912
,2019-02-08 23:50:53,1.132 sec,5.0,1522.0,1758472.7083500
,2019-02-08 23:50:53,1.174 sec,6.0,3710.0,1744870.7418730
,2019-02-08 23:50:53,1.201 sec,7.0,9178.0,1714168.6983033
,2019-02-08 23:50:53,1.228 sec,8.0,12886.0,1671183.7363382
,2019-02-08 23:50:53,1.317 sec,9.0,14173.0,1631656.6814824


In [13]:
# join cluster labels to original data for further analysis
labels = clusters.predict(frame)
labeled_frame = frame.cbind(labels)
labeled_frame[-1].head()

kmeans prediction progress: |█████████████████████████████████████████████| 100%


predict
0
0
0
0
0
0
0
0
0
2


In [14]:
# profile clusters by means
grouped = labeled_frame.group_by(by=['predict'])
means = grouped.mean(col=reals).get_frame()
print(means)

predict,mean_loan_amnt,mean_int_rate,mean_emp_length,mean_annual_inc,mean_dti,mean_delinq_2yrs,mean_revol_util,mean_total_acc,mean_longest_credit_length
0,-0.448524,-0.172252,-0.261724,-0.250118,-0.123311,-0.145322,-0.0872035,-0.417512,-0.368099
1,0.979539,-0.406632,0.793992,92.01,-2.02388,0.15268,-0.561096,0.0930262,-0.410857
2,0.738868,0.283832,0.431126,0.403167,0.203356,0.239409,0.143726,0.687861,0.606499


In [15]:
# profile clusters by modes
grouped = labeled_frame.group_by(by=['predict'])
modes = grouped.mode(col=enums).get_frame()
print(modes)

predict,mode_term,mode_home_ownership,mode_purpose,mode_addr_state,mode_verification_status
0,0,5,2,4,1
1,0,1,2,4,1
2,0,1,2,4,1


In [16]:
# define a function for plotting clusters in 2-d
def plot(_2d_labeled_frame):

    _0 = plt.scatter(features_pandas[_2d_labeled_frame.label == 0].iloc[0:750, 0], 
                     features_pandas[_2d_labeled_frame.label == 0].iloc[0:750, 1],
                     color='m', marker='^', alpha=.15)

    _1 = plt.scatter(features_pandas[_2d_labeled_frame.label == 1].iloc[0:750, 0], 
                     features_pandas[_2d_labeled_frame.label == 1].iloc[0:750, 1],
                     color='c', alpha=.15)

    _2 = plt.scatter(features_pandas[_2d_labeled_frame.label == 2].iloc[0:750, 0], 
                     features_pandas[_2d_labeled_frame.label == 2].iloc[0:750, 1],
                     color='g', marker='s', alpha=.15)               
                     
    plt.legend([_0, _1, _2], 
               ['Cluster 0', 'Cluster 1', 'Cluster 2'],
               bbox_to_anchor=(1.05, 0.0), 
               loc=3, borderaxespad=0.)
    
    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')

In [17]:
# project training data onto 2-D using principal components
# join with clusters labels
# plot
pca = H2OPrincipalComponentAnalysisEstimator(k=2) # project onto 2 PCs
pca.train(x=reals, training_frame=labeled_frame)
features = pca.predict(labeled_frame)
features_pandas = features.as_data_frame()
features_pandas['label'] = labeled_frame[-1].as_data_frame()
print(features_pandas.head())
plot(features_pandas)

# view detailed results at http://ip:port/flow/index.html

pca Model Build progress: |███████████████████████████████████████████████| 100%


/Users/bear/anaconda/lib/python3.6/site-packages/h2o/job.py:69: UserWarning: _train: Dataset used may contain fewer number of rows due to removal of rows with NA/missing values.  If this is not desirable, set impute_missing argument in pca call to TRUE/True/true/... depending on the client language.
  warnings.warn(w)


pca prediction progress: |████████████████████████████████████████████████| 100%
        PC1       PC2  label
0 -0.055350  0.839534      0
1  3.112822 -0.793804      0
2  1.266717  1.537850      0
3 -0.407435 -0.792267      0
4  2.601477 -0.798351      0


NameError: name 'plt' is not defined

In [ ]:
# shutdown h2o
# be careful - this can erase your work
h2o.cluster().shutdown(prompt=True)